# Preprocess images

To make model training more efficient and facilitate our many experiments, we perform some preprocessing on the images so that this does not need to be repeated for each experiment. The preprocessed versions are saved as .npy files.

For the detector we preprocess to obtain three types of input:
* 256x256 partitions of Y-component
* 256x256 partitions of DCT coefficients
* 64x120 DCT histogram features

For the restorer we only preprocess to obtain 256x256 RGB partitions.

## Acknowledgements

We obtain DCT histogram features using a method developed by Barni et al., which can be read in their paper: https://arxiv.org/abs/1708.00930

We use the implementation of Barni's method as developed by Park et al., which is found here: https://github.com/plok5308/DJPEG/blob/master/DJPEG_net.py

## Preparation

In [1]:
# Imports
from PIL import JpegImagePlugin
from PIL import Image
import scipy.fftpack
import numpy as np
from numpy import r_
from numpy import pi
import matplotlib.pyplot as plt
import tensorflow as tf
import cv2
import math
from multiprocessing.dummy import Pool
import threading
import sys
from itertools import repeat
from google.colab import drive

# Connect google drive and set default path
drive.mount('/content/drive')

# IMPORTANT: Change this to your own path, ending with "dataset/"
PATH = '/content/drive/MyDrive/dataset/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Detector




### Y-component pixel partitioning

We partition the images in the dataset into blocks of 256x256 of the Y-component

In [ ]:
SQUARE_SIZE = 256


def get_image_squares(x, compress="s", SQF=50, DQF=""):
    '''Convert x^th image in dataset into blocks of 256x256 of the Y channel in 
    YCbCr'''

    imgs = []

    # Download image
    try:
        img = np.array(Image.open(
            PATH + f"raise/{compress}jpeg{SQF}{DQF}/image_{x}.jpg"))
    except:
        print(f"Can't download {x}")

    # Get Y-component only
    img = cv2.cvtColor(img, cv2.COLOR_RGB2YUV)
    img = img[:, :, :1]

    #  Partition the image
    for i in range(0, len(img)-SQUARE_SIZE, SQUARE_SIZE):
        for j in range(0, len(img[i])-SQUARE_SIZE, SQUARE_SIZE):
            imgs.append((img[i:i+SQUARE_SIZE, j:j+SQUARE_SIZE]))

    return imgs

Preprocess the images and save as .npy files. Store partitions from 200 images in one file (to handle RAM constraints).

In [ ]:
# Preprocess and save all the images
for top in range(0, 200, 1000):
    for cat in ["no", "single", "double"]:
        for QF in [10, 30, 50, 70, 90]:

            # Settings
            if cat == "single":
                SQF = QF
                DQF = ""
            elif cat == "double":
                SQF = 50
                DQF = QF
            else:  # no
                if QF > 10:
                    continue  #  Only process no compression once
                SQF = ""
                DQF = ""

            print(f"Processing {top}-{top+200} {cat} compression with QF=({SQF},{DQF})")

            # Download and partition images with a threadpool for efficiency
            with Pool(threading.active_count()) as p:
                res = p.starmap(get_image_squares, zip(
                    range(top, top+200), repeat(cat[0]), repeat(SQF), repeat(DQF)))

            #  Flatten array of images
            imgs = np.array(sum(res, []))
            del res

            #  Save blocks
            np.save(
                PATH + f"detector_npy/{cat[0]}jpeg{SQF}{DQF}/{cat}blockimg{top+200}.npy", imgs)

### DCT coefficients of partitions

We also store a version of the 256x256 blocks in the DCT domain, using the DCT2 algorithm on each of the 256x256 Y-component blocks.

In [ ]:
def dct2(a):
    '''Returns the DCT2 of image block'''
    return scipy.fftpack.dct(
        scipy.fftpack.dct(a, axis=0, norm='ortho'), axis=1, norm='ortho')


def idct2(a):
    '''Returns the inverse DCT2 of image block'''
    return scipy.fftpack.idct(
        scipy.fftpack.idct(a, axis=0, norm='ortho'), axis=1, norm='ortho')


def dct8x8(im):
    '''Computes DCT2 for each 8x8 block of the image and returns the resulting 
    coefficients'''

    #  Get Y-component only
    im = im[:, :, 0]
    imsize = im.shape
    dct = np.zeros(imsize)

    # Do 8x8 DCT on image (in-place)
    for i in r_[:imsize[0]:8]:
        for j in r_[:imsize[1]:8]:
            dct[i:(i+8), j:(j+8)] = dct2(im[i:(i+8), j:(j+8)])

    return dct[..., np.newaxis]

Preprocess the images and save as .npy files. Store partitions from 200 images in one file (to handle RAM constraints).

In [ ]:
# Preprocess and save all the images
for top in ["200", "400", "600", "800", "1000"]:
    for cat in ["no", "single", "double"]:
        for QF in [10, 30, 50, 70, 90]:

            # Settings
            if cat == "single":
                SQF = QF
                DQF = ""
            elif cat == "double":
                SQF = 50
                DQF = QF
            else:  # no
                if QF > 10:
                    continue  #  Only process no compression once
                SQF = ""
                DQF = ""

            print(f"Processing {top} {cat} compression with QF=({SQF},{DQF})")

            # Load image partitions
            imgs = np.load(
                PATH + f"detector_npy/{cat[0]}jpeg{SQF}{DQF}/{cat}blockimg{top}.npy")

            #  Process each image partition one-by-one
            for i in range(len(imgs)):
                imgs[i] = dct8x8(imgs[i])

            np.save(
                PATH + f"detector_npy/{cat[0]}jpeg{SQF}{DQF}/{cat}blockdct{top}.npy", imgs)

### DCT histogram features of partitions

We process each of the 256x256 Y-component blocks, converting it to 64x120 DCT histogram features, as proposed by Barni et al., and as implemented by Park et al.

In [2]:
# Parts taken from https://github.com/plok5308/DJPEG/blob/master/DJPEG_net.py
# Calculates the DCT histogram features using convolutional layers

def cal_scale(p, q):
    if p == 0:
        ap = 1/(math.sqrt(8))
    else:
        ap = math.sqrt(0.25)
    if q == 0:
        aq = 1/(math.sqrt(8))
    else:
        aq = math.sqrt(0.25)

    return ap, aq


def cal_basis(p, q):
    basis = np.zeros((8, 8))
    ap, aq = cal_scale(p, q)
    for m in range(0, 8):
        for n in range(0, 8):
            basis[m, n] = ap*aq * \
                math.cos(math.pi*(2*m+1)*p/16)*math.cos(math.pi*(2*n+1)*q/16)

    return basis


def load_DCT_basis_64():
    basis_64 = np.zeros((8, 8, 64))
    idx = 0
    for i in range(8):
        for j in range(8):
            basis_64[:, :, idx] = cal_basis(i, j)
            idx = idx + 1
    return basis_64


def conv2d_8(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 8, 8, 1], padding="SAME")


DCT_basis_64 = load_DCT_basis_64()
np_basis = np.zeros((8, 8, 1, 64))
for i in range(64):
    np_basis[:, :, 0, i] = DCT_basis_64[:, :, i]

w_basis = tf.constant(np_basis)

Process image partitions using Barni et al. method. We process only 15000 at a time due to RAM constraints. Each 15000 are stored in a separate files, which we later combine.

In [ ]:
# Preprocess and save all the images
for top in ["200", "400", "600", "800", "1000"]:
    for cat in ["no", "single", "double"]:
        for QF in [10, 30, 50, 70, 90]:

            # Settings
            if cat == "single":
                SQF = QF
                DQF = ""
            elif cat == "double":
                SQF = 50
                DQF = QF
            else:  # no
                if QF > 10:
                    continue  #  Only process no compression once
                SQF = ""
                DQF = ""

            print(f"Processing {top} {cat} compression with QF=({SQF},{DQF})")

            # Load image partitions
            imgs = np.load(
                PATH + f"detector_npy/{cat[0]}jpeg{SQF}{DQF}/{cat}blockimg{top}.npy")

            # Process image partitions using Barni et al. method
            # We process only 15000 at a time due to RAM constraints
            for i in range(0, 45000, 15000):
                x2 = conv2d_8(imgs[i:i+15000], w_basis)  # Nx32x32x64

                gamma = 1e+06
                for b in range(-60, 61):
                    x3 = tf.divide(tf.reduce_sum(
                        tf.sigmoid(tf.scalar_mul(gamma, tf.subtract(x2, b))), [1, 2]), 1024)
                    x3 = tf.reshape(x3, [-1, 1, 64])

                    if b == -60:
                        x4 = x3
                    else:
                        x4 = tf.concat([x4, x3], 1)

                x5 = x4[:, 0:120, :] - x4[:, 1:121, :]
                x6 = tf.reshape(x5, [-1, 120, 64, 1])  # Input for CNN

                np.save(
                    PATH + f"detector_npy/{cat[0]}jpeg{SQF}{DQF}/{cat}blockcnn{top}_{i//15000 + 1}.npy", x6.numpy())

Combine the preprocessed files. Store partitions from 200 images in one file (to handle RAM constraints).

In [ ]:
# Combine files
for top in ["200", "400", "600", "800", "1000"]:
    for cat in ["no", "single", "double"]:
        for QF in [10, 30, 50, 70, 90]:

            # Settings
            if cat == "single":
                SQF = QF
                DQF = ""
            elif cat == "double":
                SQF = 50
                DQF = QF
            else:  # no
                if QF > 10:
                    continue  #  Only process no compression once
                SQF = ""
                DQF = ""

            #  Combine preprocessed files
            try:
                imgs = np.load(
                    PATH + f"detector_npy/{cat[0]}jpeg{SQF}{DQF}/{cat}blockcnn{top}_1.npy")
                for i in range(2, 4):
                    imgs = np.concatenate((imgs, np.load(
                        PATH + f"detector_npy/{cat[0]}jpeg{SQF}{DQF}/{cat}blockcnn{top}_{i}.npy")))
                np.save(
                    PATH + f"detector_npy/{cat[0]}jpeg{SQF}{DQF}/{cat}blockcnn{top}.npy", imgs)
            except:
                print(f"Could not combine {top}, {cat}")

## Restorer

### RGB pixel partitioning

As opposed to the detector, we use the coloured image (RGB) rather than a grayscale version. We also only partition once per image, due to RAM constraints in training.

In [2]:
SQUARE_SIZE = 256


def get_rgb_image_square(x, compress="s", SQF=50, DQF=""):
    '''Convert x^th image in dataset into blocks of 256x256, including all RGB
    channels'''

    #  Download image
    try:
        img = np.array(Image.open(
            PATH + f"raise/{compress}jpeg{SQF}{DQF}/image_{x}.jpg"))
    except:
        print(f"Can't download {x}")

    #  Partition the image, ensuring to make 8x8 blocks aligned
    start_x = next_n((len(img) // 2) - (SQUARE_SIZE // 2), 8)
    start_y = next_n((len(img[0]) // 2) - (SQUARE_SIZE // 2), 8)

    imgs = [img[
        start_x: start_x+SQUARE_SIZE,
        start_y: start_y+SQUARE_SIZE
    ]]

    return imgs


def next_n(num, n):
    '''Returns num rounded to the next multiple of n'''
    return n * ((num-1) // n) + n

Preprocess the images and save as .npy files.

In [ ]:
# Preprocess and save all the images
for cat in ["no", "single", "double"]:
    for QF in [10, 30, 50, 70]:

        # Settings
        if cat == "single":
            SQF = QF
            DQF = ""
        elif cat == "double":
            SQF = 50
            DQF = QF
        else:  # no
            if QF > 10:
                continue  #  Only process no compression once
            SQF = ""
            DQF = ""

        imgs = None

        #  Download and partition images with a threadpool for efficiency
        #  Process 200 at a time due to RAM constraints
        for i in range(0, 1000, 200):
            with Pool(threading.active_count()) as p:
                t = p.starmap(get_rgb_image_square, zip(
                    range(i, i+200), repeat(cat[0]), repeat(SQF), repeat(DQF)))
                t = sum(t, [])
            if imgs is not None:
                imgs = np.concatenate((imgs, t))
            else:
                imgs = t

        #  Save all blocks
        np.save(PATH + f"restorer_npy/{cat[0]}jpeg{SQF}{DQF}lite.npy", imgs)